# 정제

In [2]:
import pandas as pd

## 1. 리뷰

In [3]:
rdr_review = pd.read_csv("31-0. RDR2_reviews.csv", encoding="utf-8-sig")

In [4]:
import re
rdr_review["닉네임"] = rdr_review["닉네임"].str.replace("https://steamcommunity.com/", "", regex=False)
rdr_review["닉네임"] = rdr_review["닉네임"].str.extract(r'(?:id/|profiles/)([^/]+)')

In [5]:
rdr_review["작성일"] = rdr_review["작성일"].str.replace("Posted: ", "", regex=False)

In [6]:
rdr_review["본문"] = rdr_review["본문"].str.replace(r'^Posted:.*?\n', '', regex=True)

In [8]:
def parse_date(x):
    try:
        # day month 형식 (ex: "13 July")
        return pd.to_datetime(x + " 2025", format="%d %B %Y")
    except:
        try:
            # month day 형식 (ex: "June 19")
            return pd.to_datetime(x + " 2025", format="%B %d %Y")
        except:
            return pd.NaT

rdr_review["작성일"] = rdr_review["작성일"].apply(parse_date).dt.strftime("%Y-%m-%d")

In [9]:
rdr_review['추천 여부'] = rdr_review['추천 여부'].replace({'Not Recommended': 0, 'Recommended': 1})

C:\Users\user\AppData\Local\Temp\ipykernel_352\2710779190.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  rdr_review['추천 여부'] = rdr_review['추천 여부'].replace({'Not Recommended': 0, 'Recommended': 1})


In [11]:
import re

def clean_text(text):
    if pd.isna(text):
        return ""
    # 줄바꿈, 탭 제거
    text = re.sub(r'\s+', ' ', str(text))
    # 특수문자 제거 (영문/숫자/공백만 남기기)
    text = re.sub(r'[^A-Za-z0-9 ]+', ' ', text)
    # 한 글자 단어 제거 (a, I 제외)
    text = ' '.join([w for w in text.split() if len(w) > 1 or w.lower() in ['i', 'a']])
    return text.strip()

rdr_review["본문"] = rdr_review["본문"].apply(clean_text)

In [12]:
# 빈 행 (NaN + 공백) 삭제
rdr_review = rdr_review[~(rdr_review["본문"].isna() | rdr_review["본문"].str.strip().eq(""))]

# 인덱스 리셋
rdr_review = rdr_review.reset_index(drop=True)

In [ ]:
from googletrans import Translator
import time

# 번역기 초기화
translator = Translator()

# 번역 결과 저장
translated_results = []

for idx, row in rdr_review.iterrows():
    text = row["본문"]

    if pd.isna(text) or not str(text).strip():
        translated_text = ""
    else:
        try:
            result = translator.translate(text, src='auto', dest='en')
            translated_text = result.text.strip()
        except Exception as e:
            print(f"[에러] {idx}번 번역 실패: {e}")
            translated_text = "[번역 실패]"
    
    print(f"{idx+1}/{len(rdr_review)} | {row['닉네임']} → {translated_text}")
    translated_results.append(translated_text)

    time.sleep(0.5)  # API 차단 방지 대기

# ds_reviews에 바로 열 추가
rdr_review["본문_번역"] = translated_results

# 전체 DataFrame 저장
rdr_review.to_csv("34-1. RDR2_reviews_정제.csv", index=False, encoding='utf-8-sig')
print("번역 열 추가 및 저장 완료 → 34-1. RDR2_reviews_정제.csv")

1/10625 | 76561199030926857 → 1899
2/10625 | 76561198410842299 → Horse get white color.
3/10625 | 76561199617347293 → 1899 19 America Undone
4/10625 | 76561198434241938 → An unforgettable masterpiece that redefines what a video game can be Red Dead Redemption is more than just a game it a living breathing world filled with heart soul and unmatched attention to detail From the very first chapter you re pulled into a gripping narrative that slowly evolves into something truly unforgettable Arthur Morgan isn just a protagonist he a fully realized character whose journey is emotional powerful and painfully human You don just play the game you live in it The forests feel wild the towns feel lived in and the sunsets Unreal Whether you re hunting in the mountains fishing at dawn or just riding your horse through the countryside with the music kicking in at the perfect moment every little detail adds up to something magical The bond you build with the Van der Linde gang makes the story hit eve

In [1]:
import pandas as pd

In [3]:
rdr_review = pd.read_csv("34-1. RDR2_reviews_정제.csv", encoding='utf-8-sig')
rdr_review = rdr_review.drop(columns=["본문"])[["닉네임", "작성일", "본문_번역", "추천 여부", "플레이 시간"]]
rdr_review.to_csv("34-1. RDR2_reviews_정제.csv", index=False, encoding='utf-8-sig')
print("번역 열 추가 및 저장 완료 → 34-1. RDR2_reviews_정제.csv")

번역 열 추가 및 저장 완료 → 34-1. RDR2_reviews_정제.csv
